In [1]:
import numpy as np

from collections import defaultdict

from scipy.special import gamma
from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree

import pickle
import string
digs = string.digits + string.ascii_letters

### Data

In [2]:
class RungeKutta:
    """
    Implementation of 4th order Runge-Kutta integration
    """

    def __init__(self, beta, rho, sigma, dt):
        self.beta = beta
        self.rho = rho
        self.sigma = sigma
        self.dt = dt

    def deriviation_step(self, initial_state, derivative, dt):
        """
        Compute one evaluation step
        """

        # evaluation of state
        state = {}

        if not derivative:
            state["x"] = initial_state["x"]
            state["y"] = initial_state["y"]
            state["z"] = initial_state["z"]
        else:
            state["x"] = initial_state["x"] + derivative["dx"] * dt
            state["y"] = initial_state["y"] + derivative["dy"] * dt
            state["z"] = initial_state["z"] + derivative["dz"] * dt

        # evaluation of derivative
        derivative_next_step = {}

        derivative_next_step["dx"] = self.sigma * (state["y"] - state["x"])
        derivative_next_step["dy"] = self.rho * state["x"] - state["y"] - state["x"] * state["z"]
        derivative_next_step["dz"] = state["x"] * state["y"] - self.beta * state["z"]

        return derivative_next_step

    def _integration_step(self, state, dt):
        """
        Runge-Kutta integration of the 4th order at a time `t` with a state `state`
        with the step `dt`
        """
        # Prepare 1,2,3,4 - order derivatives for the final "best" derivative,
        # gained as 4 first elements of the Taylor's approximation

        # Random initialization of defivatives(probably will have to moove)
        derivative = dict({"dx": np.random.normal(),
                           "dy": np.random.normal(),
                           "dz": np.random.normal()})
        rk1 = self.deriviation_step(initial_state=state,
                                    derivative=None,
                                    dt=dt * 0)

        rk2 = self.deriviation_step(initial_state=state,
                                    derivative=rk1,
                                    dt=dt * 0.5
                                    )

        rk3 = self.deriviation_step(initial_state=state,
                                    derivative=rk2,
                                    dt=dt * 0.5)

        rk4 = self.deriviation_step(initial_state=state,
                                    derivative=rk3,
                                    dt=dt)

        # When all derivatives are ready, it's time to construct Rung-Kutta derivative
        # !!!! DOUBLE CHECK THE TAYLOR'S APPROXIMATIONS !!!!
        dxdt = (1 / 6) * (rk1["dx"] + 2 * rk2["dx"] + 2 * rk3["dx"] + rk4["dx"])
        dydt = (1 / 6) * (rk1["dy"] + 2 * rk2["dy"] + 2 * rk3["dy"] + rk4["dy"])
        dzdt = (1 / 6) * (rk1["dz"] + 2 * rk2["dz"] + 2 * rk3["dz"] + rk4["dz"])

        state["x"] = state["x"] + dxdt * dt
        state["y"] = state["y"] + dydt * dt
        state["z"] = state["z"] + dzdt * dt
        return state

    def get_series(self, n_iterations, initial_state=None):
        """
        Does a series of integration steps to get a series
        :return: numpy array of series
        """
        if not initial_state:
            initial_state = dict({"x": 0.62225717,
                                  "y": -0.08232857,
                                  "z": 30.60845379})

        ode_solutions = []
        for iteration in range(0, n_iterations):
            state_t = self._integration_step(initial_state, dt=self.dt)
            ode_solutions.append(list(state_t.values()))
        result = np.array(ode_solutions)
        return result[:, 0]


def reconstruct_lorenz(ts: np.ndarray, template: np.ndarray):
    ts_list = [ts[:-np.sum(template)].reshape(-1, 1)]

    for offset in template.cumsum()[:-1]:
        offset_ts = ts[offset:-(template.sum() - offset)].reshape(-1, 1)
        ts_list.append(offset_ts)
    ts_list.append(ts[np.sum(template):].reshape(-1, 1))
    reconstructed_ts = np.concatenate(ts_list, axis=1)
    return reconstructed_ts

### Template

In [3]:
def int2base(x, base):
    if x < 0:
        sign = -1
    elif x == 0:
        return digs[0]
    else:
        sign = 1

    x *= sign
    digits = []

    while x:
        digits.append(digs[int(x % base)])
        x = int(x / base)

    if sign < 0:
        digits.append('-')

    digits.reverse()

    return ''.join(digits)

class TemplateManager:

    def __init__(self, template_size, max_template_distance, min_template_distance):
        """

        :param max_template_distance: Max number of points between two nearest points in a template
        :param min_template_distance: Min number of points between two nearest points in a template
        :param template_size: Number of points in a template <-> sizeof the z-vector
        """
        self.template_size = template_size
        self.max_template_distance = max_template_distance
        self.min_template_distance = min_template_distance
        self.current_template = 0
        self.current_template_name = ("0_" * self.template_size)[:-1]

    def set_current_template(self, new_current_template):
        self.current_template = new_current_template

    def get_current_template_name(self):
        return self.current_template_name

    def _template_from_str_to_int(self, template):
        """
        String representation of template to integer representation of templete
        :Str, List[str] template: List with strings, which represent distance between point.
                        e.g. ["0", "a", "9", "b"] will result in [0, 10, 9, 11]
        :return: List[int] template
        """
        if type(template) is list:
            template = [digs.find(element) for element in template]
        elif type(template) is str:
            template = template.rjust(self.template_size, "0")
            template = list(template)
            template = [digs.find(element) for element in template]
        else:
            raise Exception("-> Template of wrong type %s" % type(template))
        return template

    def _template_from_int_to_str(self, template):
        """
        Converts the int or list of int's to a list of strings
        :param template: int / list[int]
        :return: list[str]
        """
        if type(template) is int:
            template = int2base(x=template, base=self.max_template_distance + 1)
            template = template.rjust(self.template_size, "0")
            template = list(template)
        elif type(template) is list:
            template = [int2base(element, base=self.max_template_distance + 1) for element in template]
        self.current_template_name = "_".join(template)
        return template

    def _check_templates_validity(self, template):
        """
        Check if templates satisfies the borders for the template distances.
        :List[int] template: template with integers and template distances
        :return: True/False
        """

        def check_element(x):
            return (x >= self.min_template_distance) & (x <= self.max_template_distance)

        checked_template = list(map(check_element, template))
        if all(checked_template):
            return True
        else:
            # print("Element doesn't fit diapason %i-%i" % (self.min_template_distance, self.max_template_distance))
            return False

    def next_planned_template(self, method="concurrent", step=5):
        """

        :strr method: object
        :return:
        """
        assert (type(step) is int) & (step >= 1), "Template step is not integer or not big enough "
        # TODO: Repeat while the template satisfies the conditions
        next_template = self._template_from_int_to_str(self.current_template)
        next_template = self._template_from_str_to_int(next_template)
        # Skip those templates, which does not satisfies the specification.
        while not self._check_templates_validity(template=next_template):

            if method == "random":
                next_template = np.random.random_integers(low=self.min_template_distance,
                                                          high=self.max_template_distance,
                                                          size=self.template_size - 1)
            elif method == "concurrent":
                self.current_template = self.current_template + step
                if self.current_template > (self.max_template_distance + 1) ** self.template_size:
                    print("-> Run out of templates")
                    return False
                next_template = self._template_from_int_to_str(self.current_template)
                next_template = self._template_from_str_to_int(next_template)
                # print("--> ", next_template, self._check_templates_validity(template=next_template))
            else:
                raise Exception("-> Unspecified method to generate templates. Must be \"random\" or \"concurrent\"")
        self.current_template = self.current_template + step
        return next_template

    def next_planned_significance(self):
        """

        :return:
        """
        return 0.1

    def next_planned_neighbors(self):
        """
        tm = TemplateManager(5, 10, 0)
        :return:
        """
        return 11

### Wishart

In [4]:
def construct_sorted_vertexes_matrix_r(X, wishart_neighbors):
    kdt = KDTree(X, metric='euclidean')
    distances, neighbors = kdt.query(X, k=wishart_neighbors + 1)
    neighbors = neighbors[:, 1:]
    distances = distances[:, -1]
    indexes = np.argsort(distances)
    return neighbors, distances, indexes


def find_connected_clusters(graph_vertexes, vertex_neighbors, index):
    neighbors_clusters = graph_vertexes[vertex_neighbors[index]]
    unique_clusters = np.unique(neighbors_clusters).astype(int)
    unique_clusters = unique_clusters[unique_clusters != -1]
    return unique_clusters


class WishartClassifier:
    def __init__(self, wishart_neighbors, significance_level):
        self.wishart_neighbors = wishart_neighbors  # Number of neighbors
        self.significance_level = significance_level  # Significance level

    def fit(self, X):
        neighbors, distances, indexes = construct_sorted_vertexes_matrix_r(X, self.wishart_neighbors)

        size, dim = X.shape
        self.object_labels = np.zeros(size, dtype=int) - 1
        self.clusters = np.array([(1., 1., 0)])
        self.clusters_to_objects = defaultdict(list)

        for index in indexes:

            unique_clusters = find_connected_clusters(self.object_labels, neighbors, index)

            if len(unique_clusters) == 0:
                self._create_new_cluster(index, distances[index])
            else:
                max_cluster = unique_clusters[-1]
                min_cluster = unique_clusters[0]
                if max_cluster == min_cluster:
                    if self.clusters[max_cluster][-1] < 0.5:
                        self._add_elem_to_exist_cluster(index, distances[index], max_cluster)
                    else:
                        self._add_elem_to_noise(index)
                else:
                    my_clusters = self.clusters[unique_clusters]
                    flags = my_clusters[:, -1]
                    if np.min(flags) > 0.5:
                        self._add_elem_to_noise(index)
                    else:
                        significan = np.power(my_clusters[:, 0], -dim) - np.power(my_clusters[:, 1], -dim)
                        significan *= self.wishart_neighbors
                        significan /= size
                        significan /= np.power(np.pi, dim / 2)
                        significan *= gamma(dim / 2 + 1)
                        significan_index = significan >= self.significance_level

                        significan_clusters = unique_clusters[significan_index]
                        not_significan_clusters = unique_clusters[~significan_index]
                        significan_clusters_count = len(significan_clusters)
                        if significan_clusters_count > 1 or min_cluster == 0:
                            self._add_elem_to_noise(index)
                            self.clusters[significan_clusters, -1] = 1
                            for not_sig_cluster in not_significan_clusters:
                                if not_sig_cluster == 0:
                                    continue

                                for bad_index in self.clusters_to_objects[not_sig_cluster]:
                                    self._add_elem_to_noise(bad_index)
                                self.clusters_to_objects[not_sig_cluster].clear()
                        else:
                            for cur_cluster in unique_clusters:
                                if cur_cluster == min_cluster:
                                    continue

                                for bad_index in self.clusters_to_objects[cur_cluster]:
                                    self._add_elem_to_exist_cluster(bad_index, distances[bad_index], min_cluster)
                                self.clusters_to_objects[cur_cluster].clear()
                            self._add_elem_to_exist_cluster(index, distances[index], min_cluster)

        return self.clear()

    def clear(self):
        unique = np.unique(self.object_labels)
        index = np.argsort(unique)
        if unique[0] != 0:
            index += 1
        true_cluster = {unq: index for unq, index in zip(unique, index)}
        result = np.zeros(len(self.object_labels), dtype=int)
        for index, unq in enumerate(self.object_labels):
            result[index] = true_cluster[unq]
        return result

    def _add_elem_to_noise(self, index):
        self.object_labels[index] = 0
        self.clusters_to_objects[0].append(index)

    def _create_new_cluster(self, index, dist):
        self.object_labels[index] = len(self.clusters)
        self.clusters_to_objects[len(self.clusters)].append(index)
        self.clusters = np.append(self.clusters, [(dist, dist, 0)], axis=0)

    def _add_elem_to_exist_cluster(self, index, dist, cluster_label):
        self.object_labels[index] = cluster_label
        self.clusters_to_objects[cluster_label].append(index)
        self.clusters[cluster_label][0] = min(self.clusters[cluster_label][0], dist)
        self.clusters[cluster_label][1] = max(self.clusters[cluster_label][1], dist)

In [5]:
def construct_template_dict(template_name, clusters, data_lorenz):
    unique_clusters = np.unique(clusters)
    unique_clusters = np.delete(unique_clusters, 0)
    cluster_centers = []
    cluster_sizes = []
    for cluster in unique_clusters:
        mask = clusters == cluster
        zvectors = data_lorenz[mask]
        cluster_center = zvectors.mean(0)
        cluster_centers.append(cluster_center)
        cluster_sizes.append(mask.sum())
    
    cluster_sizes = np.array(cluster_sizes)
    cluster_centers = np.array(cluster_centers)
    zvector_base = cluster_centers[:, :-1]
    predicted_values = cluster_centers[:, -1]
    
    kdtree = cKDTree(zvector_base)
    
    template_dict = {"template_name": template_name,
                     "cluster_sizes": cluster_sizes,
                     "kdtree": kdtree,
                     "predicted_values": predicted_values}
    return template_dict

### Fit data

In [6]:
rk = RungeKutta(beta=8 / 3, rho=28, sigma=10, dt=0.1)
lorenz_train = rk.get_series(n_iterations=int(3e5))
lorenz_train = lorenz_train / abs(lorenz_train.max())

In [14]:
tm = TemplateManager(template_size=4, max_template_distance=10, min_template_distance=1)
tm.current_template = 3000
STEP = 10

In [15]:
for i in range(300):
    
    template = tm.next_planned_template(step=10)
    template_name = tm.get_current_template_name()
    
    reconstructed_train_lorenz = reconstruct_lorenz(lorenz_train, np.array(template))
    
    ws = WishartClassifier(11, 0.1)
    clusters = ws.fit(reconstructed_train_lorenz)
    
    template_dict = construct_template_dict(template_name, clusters, reconstructed_train_lorenz)
    
    with open(("models/" + template_name + ".pkl"), "wb") as f:
        pickle.dump(template_dict, f)